In [1]:
!pip3 install -r requirements.txt

In [18]:
from catboost import CatBoostRegressor, Pool

import numpy as np
import pandas as pd
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [3]:
sns.set_style(style="darkgrid")

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [16]:
df0 = pd.read_csv("train.csv", delimiter=",")
X_val = pd.read_csv("test.csv", delimiter=",")
df0 = df0.rename(columns={
    "Whole weight": "Whole_weight",
    "Whole weight.1": "Shucked_weight",
    "Whole weight.2": "Viscera_weight",
    "Shell weight": "Shell_weight"
    })

X_val = X_val.rename(columns={
    "Whole weight": "Whole_weight",
    "Whole weight.1": "Shucked_weight",
    "Whole weight.2": "Viscera_weight",
    "Shell weight": "Shell_weight"
    })

df = df0.copy()
df["Rings"] = np.log1p(df["Rings"])
non_cat_features = ["Length", "Diameter", "Height", "Whole_weight", "Shucked_weight", "Viscera_weight", "Shell_weight"]
cat_features = ["Sex"]

df_order_enc = df.drop(columns=["id", "Sex"])
df_order_enc["Sex"] = np.where(df["Sex"] == "M", 1, np.where(df["Sex"] == "I", 0, -1))

whis = 1.5
Q1, Q3 = df[non_cat_features].quantile(0.25), df[non_cat_features].quantile(0.75)
IQR = Q3 - Q1
top_lim, bottom_lim = Q3 + whis * IQR, Q1 - whis * IQR

mask = (df[non_cat_features] >= bottom_lim) & (df[non_cat_features] <= top_lim)
df = df[mask.all(axis=1)]

print(f"В датасете осталось {len(df)} строк")

non_cat_features = ["Length", "Diameter", "Height", "Whole_weight", "Shucked_weight", "Viscera_weight", "Shell_weight"]
cat_features = ["Sex"]
X_train, y_train = df[non_cat_features + cat_features], df["Rings"]
X_val_clear = X_val[non_cat_features + cat_features]

В датасете осталось 87266 строк


In [13]:
train_pool = Pool(X_train, label=y_train, cat_features=cat_features)
catboost_model = CatBoostRegressor(
    loss_function="RMSE",
).fit(train_pool)

Learning rate set to 0.082953
0:	learn: 0.2575055	total: 12.9ms	remaining: 12.9s
1:	learn: 0.2464385	total: 25.4ms	remaining: 12.7s
2:	learn: 0.2364590	total: 37.3ms	remaining: 12.4s
3:	learn: 0.2277094	total: 52.2ms	remaining: 13s
4:	learn: 0.2199745	total: 65.4ms	remaining: 13s
5:	learn: 0.2133148	total: 78ms	remaining: 12.9s
6:	learn: 0.2071186	total: 90.5ms	remaining: 12.8s
7:	learn: 0.2017426	total: 101ms	remaining: 12.5s
8:	learn: 0.1968270	total: 113ms	remaining: 12.4s
9:	learn: 0.1925806	total: 125ms	remaining: 12.3s
10:	learn: 0.1888628	total: 136ms	remaining: 12.2s
11:	learn: 0.1854786	total: 147ms	remaining: 12.1s
12:	learn: 0.1824461	total: 157ms	remaining: 11.9s
13:	learn: 0.1796471	total: 168ms	remaining: 11.9s
14:	learn: 0.1772480	total: 179ms	remaining: 11.7s
15:	learn: 0.1750487	total: 190ms	remaining: 11.7s
16:	learn: 0.1731826	total: 200ms	remaining: 11.6s
17:	learn: 0.1715462	total: 209ms	remaining: 11.4s
18:	learn: 0.1700559	total: 220ms	remaining: 11.3s
19:	learn:

In [17]:
baseline = np.expm1(catboost_model.predict(X_val_clear))

with open("baseline.csv", "w") as f:
    f.write(f"id,Rings\n")
    for row_num in range(len(baseline)):
        f.write(f"{X_val["id"].iloc[row_num]},{baseline[row_num]}\n")